In [1]:
import os
import numpy as np

from dataloader import load_raw, create_datasets
from measures import compute_intermittent_indicators

datasets = ["M5","carparts","OnlineRetail","Auto","RAF"]
d = datasets[2]

data_raw, data_info = load_raw(dataset_name=d, datasets_folder_path=os.path.join("..","data"))
adi_raw, cv2_raw = compute_intermittent_indicators(data_raw, data_info['h'], na_rm = False, zero_rm=False)
adi, cv2 = compute_intermittent_indicators(data_raw, data_info['h'])

datasets = create_datasets(data_raw, data_info)

mask_nico = \
    np.logical_not(np.all(data_raw.values[:,:-data_info['h']*2]==0, axis=1)) * \
    np.logical_not((np.sum(np.isnan(data_raw.values), axis=1) > 0)) * \
    (adi_raw >= 1.32)
mask_ste = \
    np.logical_not(np.all(data_raw.values[:,:-data_info['h']]==0, axis=1)) * \
    np.logical_not((np.sum(np.isnan(data_raw.values), axis=1) > 0)) * \
    (adi_raw >= 1.32) * \
    ((~np.all(data_raw.values[:,:200] == 0, axis=1)) if d=="OnlineRetail" else True)

assert np.sum(adi>=1.32) == np.sum(mask_nico)

print('tot ', data_raw.shape[0])
print('nico', np.sum(mask_nico))
print('ste ', np.sum(mask_ste))

tot  2489
nico 2354
ste  2023


In [250]:
tsi_nico = np.arange(data_raw.shape[0])[mask_nico]
tsi_ste = np.arange(data_raw.shape[0])[mask_ste]

mask_onlineretail = (~np.isin(tsi_nico, np.setdiff1d(tsi_nico, tsi_ste))) # d must be "OnlineRetail" !

In [260]:
np.save("cache_global/mask_onlineretail.npy", mask_onlineretail)